In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_excel("tumor-data-without-healthy.xlsx")

In [3]:
labels = [value for column, value in data.iloc[0].items()]
data = data.set_axis(labels, axis=1)
data = data[data.columns[1:]].apply(
    lambda x: pd.to_numeric(x.replace('<', '').replace(',', '.'), errors='coerce')
)
data.drop(index=0, inplace=True)

In [4]:
data["Гормональная активность 0-нет               1-да"] = data[
    "Гормональная активность 0-нет               1-да"].astype(str)
data["Гормональная активность 0-нет               1-да"] = data[
"Гормональная активность 0-нет               1-да"].fillna("missing")

In [5]:
""" dropping some columns """

data.dropna(axis=1, how="all", inplace=True)
# data.dropna(axis=0, thresh=int(data.shape[1] * 0.5), inplace=True)
data.dropna(axis=1, thresh=int(data.shape[0] * 0.5), inplace=True)
data.drop(["Пол:    0-жен, 1-муж"], inplace=True, axis=1)

In [6]:
X = data.drop(columns=["КАН", "АКР"])
y = np.argmax(data[["КАН", "АКР"]], axis=1)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)

In [8]:
import optuna

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer

def objective(trial):
    param = {
        "C":trial.suggest_float('C', 1e-5, 1e5, log=True),
        "max_iter":trial.suggest_int('max_iter', 300, 2000),
        "solver":trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga']),
        "penalty":trial.suggest_categorical('penalty', ['l2']),
        'fit_intercept' : trial.suggest_categorical('fit_intercept' , [True, False]),
    }

    model = make_pipeline(
        SimpleImputer(strategy="median"),
        LogisticRegression(**param, random_state=42)
    )

    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-12-24 16:28:45,781] A new study created in memory with name: no-name-a5718b03-b06c-4368-906f-ba4216f81db8
[I 2024-12-24 16:28:45,797] Trial 0 finished with value: 0.9090909090909091 and parameters: {'C': 2.3360902161801474e-05, 'max_iter': 743, 'solver': 'liblinear', 'penalty': 'l2', 'fit_intercept': False}. Best is trial 0 with value: 0.9090909090909091.
[I 2024-12-24 16:28:45,813] Trial 1 finished with value: 0.8181818181818182 and parameters: {'C': 415.97086120088545, 'max_iter': 1609, 'solver': 'liblinear', 'penalty': 'l2', 'fit_intercept': True}. Best is trial 0 with value: 0.9090909090909091.
C:\Users\arafa\anaconda3\envs\university-projects\lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2024-12-24 16:28:45,877] Trial 2 finished with value: 0.9090909090909091 and parameters: {'C': 0.2148640832274, 'max_iter': 959, 'solver': 'saga', 'penalty': 'l2', 'fit_intercept':

In [9]:
model = make_pipeline(
    SimpleImputer(strategy="median"),
    LogisticRegression(**study.best_params)
)

In [10]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for i, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    print(f"Fold {i}:")
    xtrain, xtest = X_train.iloc[train_index], X_train.iloc[test_index]
    ytrain, ytest = y_train[train_index], y_train[test_index]
    model.fit(xtrain, ytrain)

    y_pred = model.predict(xtest)

    print("Accuracy:", accuracy_score(ytest, y_pred))
    print("F1:", f1_score(ytest, y_pred))
    print("Precision:", precision_score(ytest, y_pred))
    print("Recall:", recall_score(ytest, y_pred))
    print()

Fold 0:
Accuracy: 0.6842105263157895
F1: 0.625
Precision: 0.625
Recall: 0.625

Fold 1:
Accuracy: 0.7777777777777778
F1: 0.75
Precision: 0.75
Recall: 0.75

Fold 2:
Accuracy: 0.7777777777777778
F1: 0.75
Precision: 0.75
Recall: 0.75

Fold 3:
Accuracy: 0.7777777777777778
F1: 0.7142857142857143
Precision: 0.8333333333333334
Recall: 0.625

Fold 4:
Accuracy: 0.8888888888888888
F1: 0.875
Precision: 0.875
Recall: 0.875



In [11]:
from sklearn.model_selection import cross_validate

cv = cross_validate(model, X, y, n_jobs=-1,
                    scoring=["accuracy",
                             "f1",
                             "precision",
                             "recall"])

In [13]:
cv["test_accuracy"].mean()

0.8023809523809524

In [12]:
for i in range(len(cv)-1):
    print(
        " accuracy", cv["test_accuracy"][i], "\n",
        "f1", cv["test_f1"][i], "\n",
        "recall", cv["test_recall"][i], "\n",
        "precision", cv["test_precision"][i], "\n"
    )
    print("---------------------------------------------")

 accuracy 0.9047619047619048 
 f1 0.8888888888888888 
 recall 0.8 
 precision 1.0 

---------------------------------------------
 accuracy 0.8571428571428571 
 f1 0.8235294117647058 
 recall 0.7 
 precision 1.0 

---------------------------------------------
 accuracy 0.85 
 f1 0.8235294117647058 
 recall 0.7777777777777778 
 precision 0.875 

---------------------------------------------
 accuracy 0.7 
 f1 0.6666666666666666 
 recall 0.6666666666666666 
 precision 0.6666666666666666 

---------------------------------------------
 accuracy 0.7 
 f1 0.7 
 recall 0.7777777777777778 
 precision 0.6363636363636364 

---------------------------------------------
